In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline

/home/dro/.local/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("train_dataset_train.csv")
test = pd.read_csv('test_dataset_test.csv')

/tmp/ipykernel_661513/398589533.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("train_dataset_train.csv")
/tmp/ipykernel_661513/398589533.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv('test_dataset_test.csv')


## Преобразуем индекс операции

In [3]:
df['index_oper'] = df['index_oper'].astype(str).apply(lambda x: x if len(x) >= 4 else '0').astype(float).astype(int)
test['index_oper'] = test['index_oper'].astype(str).apply(lambda x: x if len(x) >= 4 else '0').astype(float).astype(int)

## Категоризуем некоторые столбцы

In [25]:
cols_to_categorize = ['is_in_yandex', 'is_return', 'type', 'is_privatecategory']

In [26]:
for col in cols_to_categorize:
    df[col] = pd.Categorical(df[col])
    df[col].astype('category').cat.codes
    df[col] = df[col].cat.codes
    
    test[col] = pd.Categorical(test[col])
    test[col].astype('category').cat.codes
    test[col] = test[col].cat.codes

## Готовие обучающие выборки

In [29]:
col_obj = df.select_dtypes(include=['object']).columns.values
col_obj = list(set(col_obj) ^ set(["id", "label"]))

In [30]:
col_obj

['oper_type + oper_attr', 'label', 'name_mfi', 'id']

In [31]:
X = df.drop(col_obj, axis = 1)
y = df[["label"]]

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

## Обучение модели

In [33]:
from sklearn.metrics import recall_score

def scorer(y_true, y_pred):

    return 'recall', recall_score(y_true, y_pred > 0.01, average='macro'), True

In [34]:
import lightgbm as lgb

In [35]:
clf = lgb.LGBMClassifier(num_leaves=12023, random_state=42, n_estimators=100)

In [36]:
clf.fit(X_train, y_train, eval_set=(X_test, y_test), eval_metric=scorer)

/home/dro/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dro/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[1]	valid_0's binary_logloss: 0.107354	valid_0's recall: 0.5
[2]	valid_0's binary_logloss: 0.10136	valid_0's recall: 0.5
[3]	valid_0's binary_logloss: 0.0972032	valid_0's recall: 0.5
[4]	valid_0's binary_logloss: 0.0939592	valid_0's recall: 0.5
[5]	valid_0's binary_logloss: 0.0913446	valid_0's recall: 0.5
[6]	valid_0's binary_logloss: 0.0892653	valid_0's recall: 0.5
[7]	valid_0's binary_logloss: 0.0875784	valid_0's recall: 0.5
[8]	valid_0's binary_logloss: 0.0860028	valid_0's recall: 0.5
[9]	valid_0's binary_logloss: 0.084646	valid_0's recall: 0.5
[10]	valid_0's binary_logloss: 0.0835598	valid_0's recall: 0.5
[11]	valid_0's binary_logloss: 0.0825489	valid_0's recall: 0.823847
[12]	valid_0's binary_logloss: 0.0817786	valid_0's recall: 0.830588
[13]	valid_0's binary_logloss: 0.0810976	valid_0's recall: 0.835435
[14]	valid_0's binary_logloss: 0.0804506	valid_0's recall: 0.849896
[15]	valid_0's binary_logloss: 0.07996	valid_0's recall: 0.862189
[16]	valid_0's binary_logloss: 0.0795263	vali

LGBMClassifier(num_leaves=12023, random_state=42)

## Оценка точности

In [37]:
pred = clf.predict_proba(X_test)[:, 1] > 0.007

In [38]:
from sklearn.metrics import recall_score

score = recall_score(y_test, pred, average = "macro" )

In [39]:
print("Recall", score)

Recall 0.8778555748401962


In [40]:
ss = pd.read_csv('sample_solution.csv')

In [41]:
ss['label'] = (clf.predict_proba(test[X_test.columns])[:, 1] > 0.007).astype(int)


In [42]:
ss.to_csv('solut.csv', index=False)

In [43]:
ss['label'].mean()

0.1994335